In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import json
import os
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import re
import seaborn as sns
import matplotlib.pyplot as plt
import swifter
import multiprocessing
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.formula.api as smf
from collections import defaultdict
from dataclasses import dataclass
from torch import nn
import torch.nn.functional as F
import torch
import random
import copy
from tqdm import tqdm

sns.set_style("darkgrid")

# Loading Processed Reviews

In [3]:
# Paths
PROCESSED_FOLDER = './data/processed/'
# PROCESSED_REVIEWS_FILE = 'processed_reviews.csv'
PROCESSED_REVIEWS_FILE = 'processed_reviews_with_sentiment.csv'

In [4]:
reviews = pd.read_csv(os.path.join(PROCESSED_FOLDER, PROCESSED_REVIEWS_FILE))

In [5]:
reviews.head()

,review_id,user_id,item_id,text,rating,tokenized_text,neg_sent_avg,neu_sent_avg,pos_sent_avg,compound_sent_avg
0,255938,0,1,"First things first. My ""reviews"" system is exp...",8,"[['First', 'things', 'first', '.'], ['My', '``...",0.073333,0.757708,0.168937,0.208675
1,259117,1,2,Let me start off by saying that Made in Abyss ...,10,"[['Let', 'me', 'start', 'off', 'by', 'saying',...",0.062741,0.759333,0.177963,0.302804
2,253664,2,3,"Art 9/10: It is great, especially the actions ...",7,"[['Art', '9/10', ':', 'It', 'is', 'great', ','...",0.047278,0.792889,0.159833,0.220767
3,247454,3,4,As someone who loves Studio Ghibli and its mov...,6,"[['As', 'someone', 'who', 'loves', 'Studio', '...",0.055577,0.809192,0.135269,0.187896
4,23791,4,5,code geass is one of those series that everybo...,10,"[['code', 'geass', 'is', 'one', 'of', 'those',...",0.028857,0.723000,0.248143,0.534129


# Converting Data for Modeling

In [6]:
# Convert item_id to 0 indexed
if min(reviews['item_id']) != 0:
    reviews['item_id'] = reviews['item_id'] - 1
    print("Done")

Done


In [7]:
@dataclass
class Review:
    user_id: int
    item_id: int
    rating: int
    text: str
    pos_sent: float
    neg_sent: float
    compound_sent: float

In [8]:
USER_KEY = 'user_id'
ITEM_KEY = 'item_id'
RATING_KEY = 'rating'

In [9]:
reviews.head()

,review_id,user_id,item_id,text,rating,tokenized_text,neg_sent_avg,neu_sent_avg,pos_sent_avg,compound_sent_avg
0,255938,0,0,"First things first. My ""reviews"" system is exp...",8,"[['First', 'things', 'first', '.'], ['My', '``...",0.073333,0.757708,0.168937,0.208675
1,259117,1,1,Let me start off by saying that Made in Abyss ...,10,"[['Let', 'me', 'start', 'off', 'by', 'saying',...",0.062741,0.759333,0.177963,0.302804
2,253664,2,2,"Art 9/10: It is great, especially the actions ...",7,"[['Art', '9/10', ':', 'It', 'is', 'great', ','...",0.047278,0.792889,0.159833,0.220767
3,247454,3,3,As someone who loves Studio Ghibli and its mov...,6,"[['As', 'someone', 'who', 'loves', 'Studio', '...",0.055577,0.809192,0.135269,0.187896
4,23791,4,4,code geass is one of those series that everybo...,10,"[['code', 'geass', 'is', 'one', 'of', 'those',...",0.028857,0.723000,0.248143,0.534129


In [10]:
user_to_reviews = defaultdict(list)
for _, row in reviews.iterrows():
    user_id, item_id, rating, text = row[USER_KEY], row[ITEM_KEY], row[RATING_KEY], row['text']
    pos_sent, neg_sent, compound_sent = row['pos_sent_avg'], row['neg_sent_avg'], row['compound_sent_avg']
    user_to_reviews[user_id].append(Review(user_id, item_id, rating, text, pos_sent, neg_sent, compound_sent))

## Creating the score matrix

In [11]:
# users by items
X = np.zeros(shape=(reviews['user_id'].nunique(), reviews['item_id'].nunique()))

In [12]:
for _, row in reviews.iterrows():
    user_id, item_id, rating = row[USER_KEY], row[ITEM_KEY], row[RATING_KEY]
    X[user_id][item_id] = rating

## Train/Test Split

In [13]:
train_X = copy.deepcopy(X)
valid_X = np.zeros(shape=X.shape)
test_X = np.zeros(shape=X.shape)

for user_id, reviews in user_to_reviews.items():
    # can confirm this actually shuffles properly (this code block works)
    random.shuffle(reviews)

    # Leave one out for valid
    valid_review = reviews[0]
    train_X[valid_review.user_id][valid_review.item_id] = 0
    valid_X[valid_review.user_id][valid_review.item_id] = valid_review.rating
    
    # Leave one out for test
    test_review = reviews[1]
    train_X[test_review.user_id][test_review.item_id] = 0
    test_X[test_review.user_id][test_review.item_id] = test_review.rating
    
    # Rest for train

## Creating bias terms for users / items from the training data

In [14]:
# users
user_to_pos_sent = defaultdict(list)
user_to_neg_sent = defaultdict(list)
user_to_compound_sent = defaultdict(list)

# items
item_to_pos_sent = defaultdict(list)
item_to_neg_sent = defaultdict(list)
item_to_compound_sent = defaultdict(list)

# loadding
for user_id, reviews in user_to_reviews.items():
    for r in reviews:
        # skip if not in train
        if train_X[user_id, r.item_id] == 0:
            continue
        user_to_pos_sent[user_id].append(r.pos_sent)
        user_to_neg_sent[user_id].append(r.neg_sent)
        user_to_compound_sent[user_id].append(r.compound_sent)
        item_to_pos_sent[r.item_id].append(r.pos_sent)
        item_to_neg_sent[r.item_id].append(r.neg_sent)
        item_to_compound_sent[r.item_id].append(r.compound_sent)

In [15]:
# Averaging values to get bias term
def list_mapping_to_float_mapping(hm: dict):
    id_to_sent_term = defaultdict(float)
    for k, v in hm.items():
        id_to_sent_term[k] = np.mean(v)
    return id_to_sent_term

user_to_pos_sent_term = list_mapping_to_float_mapping(user_to_pos_sent)
user_to_neg_sent_term = list_mapping_to_float_mapping(user_to_neg_sent)
user_to_compound_sent_term = list_mapping_to_float_mapping(user_to_compound_sent)
item_to_pos_sent_term = list_mapping_to_float_mapping(item_to_pos_sent)
item_to_neg_sent_term = list_mapping_to_float_mapping(item_to_neg_sent)
item_to_compound_sent_term = list_mapping_to_float_mapping(item_to_compound_sent)

## Vanilla NCF (w/ MLP)

In [265]:
def l2_regularization(values):
    return torch.sum(torch.square(values))

class VanillaNCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=20, regularization_constant=1e-6, sentiment_regularization_constant=1e-6, eps=1e-8):
        super().__init__()
        self.user_factors = nn.Embedding(num_users, embedding_dim)
        self.item_factors = nn.Embedding(num_items, embedding_dim)
        self.regularization_constant = regularization_constant
        self.sentiment_regularization_constant = sentiment_regularization_constant # unused
        self.eps = eps
        
        # NCF layers
        self.fc1 = nn.Linear(2*embedding_dim, 1)
        self.relu1 = nn.LeakyReLU()
        self.fc2 = nn.Linear(128, 10)
        self.relu2 = nn.LeakyReLU()
        self.fc3 = nn.Linear(10, 1)
        
        
    def forward(self, users: torch.LongTensor, items: torch.LongTensor):
        # user is shape (users, 1)
        # item is shape (items, 1)
        # embedding output shape is (*, emb_dim) = (users/items, emb_dim)
        user_latent_factors = self.user_factors(users)
        item_latent_factors = self.item_factors(items)
        
        # Concat latent facts together => (*, 2*emb_dim)
        user_item_latent_factors = torch.cat((user_latent_factors, item_latent_factors), dim=1)

        # FC takes (*, in_dim) and outputs (*, out_dim)
        output = self.fc1(user_item_latent_factors)
        output = self.relu1(output)
        output = self.fc2(output)
        output = self.relu2(output)
        output = self.fc3(output)
        
        # Clip in the desired range
        pred_ratings = 1 + 9 * torch.sigmoid(output).squeeze()  

        return pred_ratings
    
    def loss(self, pred_rating: torch.LongTensor, rating: torch.LongTensor, rmse=False):
        if rmse:
            loss = torch.sqrt(F.mse_loss(pred_rating, rating) + self.eps)
        else:
            loss = F.mse_loss(pred_rating, rating) + self.eps
        
        
        # L2 Regularization
        sum_of_squared_values = l2_regularization(self.user_factors.weight) + l2_regularization(self.item_factors.weight)
        l2_penalty = (1/len(rating)) * self.regularization_constant * sum_of_squared_values
        
        # Total Loss
        total_loss = loss + l2_penalty
        return total_loss
    
    def predict_single_interaction(self, user_id: int, item_id: int):
        user = torch.LongTensor([user_id]).cuda()
        item = torch.LongTensor([item_id]).cuda()
        return self.forward(user, item)

In [199]:
def eval_MSE_loss(eval_X, model, round_digits=3):
    """Uses reduction mean"""
    user_ids_list, item_ids_list = eval_X.nonzero()
    gt_ratings = torch.FloatTensor([eval_X[user_id, item_id] for user_id, item_id in zip(user_ids_list, item_ids_list)]).to(device)
    curr_users_tensor = torch.LongTensor(user_ids_list).to(device)
    curr_items_tensor = torch.LongTensor(item_ids_list).to(device)
    pred_ratings = model.forward(curr_users_tensor, curr_items_tensor)
    
    return round(F.mse_loss(pred_ratings, gt_ratings).item(), 3)

def eval_RMSE_loss(eval_X, model):
    """Uses reduction mean"""
    user_ids_list, item_ids_list = eval_X.nonzero()
    gt_ratings = torch.FloatTensor([eval_X[user_id, item_id] for user_id, item_id in zip(user_ids_list, item_ids_list)]).to(device)
    curr_users_tensor = torch.LongTensor(user_ids_list).to(device)
    curr_items_tensor = torch.LongTensor(item_ids_list).to(device)
    pred_ratings = model.forward(curr_users_tensor, curr_items_tensor)
    
    return round(torch.sqrt(F.mse_loss(pred_ratings, gt_ratings)).item(), 3)

def eval_MAE_loss(eval_X, model):
    """Uses reduction mean"""
    user_ids_list, item_ids_list = eval_X.nonzero()
    gt_ratings = torch.FloatTensor([eval_X[user_id, item_id] for user_id, item_id in zip(user_ids_list, item_ids_list)]).to(device)
    curr_users_tensor = torch.LongTensor(user_ids_list).to(device)
    curr_items_tensor = torch.LongTensor(item_ids_list).to(device)
    pred_ratings = model.forward(curr_users_tensor, curr_items_tensor)
    
    return round(F.l1_loss(pred_ratings, gt_ratings).item(), 3)


def train_v2(train_X, valid_X, model, optimizer, n_epochs=10, batch_size=5, rmse=False):
    """Training Function, calculates training and validation loss"""
    
    for epoch in (range(1, n_epochs+1)):
        users, items = train_X.nonzero()
        num_examples = len(users)
        permuted_indices = np.random.permutation(num_examples)
        users, items = users[permuted_indices], items[permuted_indices]
        

        total_train_loss = 0
        
        
        for i in tqdm(range(num_examples // batch_size)):
            user_ids_list = users[i*batch_size:i*batch_size+batch_size]
            item_ids_list = items[i*batch_size:i*batch_size+batch_size]

            # Set gradients to zero
            optimizer.zero_grad()

            # Turn data into tensors
            ratings = torch.FloatTensor([train_X[user_id, item_id] for user_id, item_id in zip(user_ids_list, item_ids_list)]).to(device)
            curr_users_tensor = torch.LongTensor(user_ids_list).to(device)
            curr_items_tensor = torch.LongTensor(item_ids_list).to(device)

            # Predict and calculate loss
            pred_ratings = model.forward(curr_users_tensor, curr_items_tensor)
            assert pred_ratings.shape == ratings.shape
            
            ## SELECTING LOSS HERE
            # loss = model.loss(pred_rating, rating)
            loss = model.loss(pred_ratings, ratings, rmse=rmse)

            # Backpropagate
            loss.backward()

            # Update the parameters
            optimizer.step()

            # MSE Loss w/o regularization (just for status updates)
            total_train_loss += F.mse_loss(pred_ratings, ratings, reduction='sum')

        # Computing validation loss for display
        total_valid_loss = eval_MSE_loss(valid_X, model)
        total_valid_RMSE_loss = eval_RMSE_loss(valid_X, model)
        total_valid_MAE_loss = eval_MAE_loss(valid_X, model)
        
        print(f"Epoch {epoch} MSE Loss: {round(total_train_loss.item() / (batch_size * (num_examples//batch_size)), 3)}, valid MSE Loss: {total_valid_loss}, valid RMSE Loss: {total_valid_RMSE_loss}, valid MAE Loss: {total_valid_MAE_loss}")

## Config Cuda

In [ ]:
!nvidia-smi

In [259]:
## Forcing GPU
assert torch.cuda.is_available()
torch.cuda.set_device("cuda:5")
device = torch.device("cuda")
a = torch.tensor([[1., 2.], [3., 4.]]).to(device)
a

tensor([[1., 2.],
        [3., 4.]], device='cuda:5')

## Training

In [276]:
# these parameter settings are pretty good, just adjust LR after you get low enough => MSE of 3.6)
# batch_size=64
# Adam
# weight decay in regularization constant
embedding_dim=200
lr=1e-2
regularization_constant=.011
sentiment_regularization_constant=0

model = VanillaNCF(num_users=X.shape[0], num_items=X.shape[1], 
                    embedding_dim=embedding_dim, 
                    regularization_constant=regularization_constant,
                    sentiment_regularization_constant=sentiment_regularization_constant
                   )
if device.type == 'cuda':
    model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [277]:
model.predict_single_interaction(0,0)

tensor(5.8509, device='cuda:5', grad_fn=<AddBackward0>)

### On-the-fly-modifications (should be commented out when you first start training)

In [270]:
# # # On-the-fly modifications
lr = 1e-3
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# model.regularization_constant = .01

In [278]:
train_v2(train_X, valid_X, model, optimizer, n_epochs=25, batch_size=64, rmse=True)

100%|████████████████████████████████| 508/508 [00:01<00:00, 304.37it/s]


Epoch 1 MSE Loss: 3.892, valid MSE Loss: 3.426, valid RMSE Loss: 1.851, valid MAE Loss: 1.44


100%|████████████████████████████████| 508/508 [00:01<00:00, 340.08it/s]


Epoch 2 MSE Loss: 3.114, valid MSE Loss: 3.233, valid RMSE Loss: 1.798, valid MAE Loss: 1.379


100%|████████████████████████████████| 508/508 [00:01<00:00, 336.92it/s]


...

Epoch 23 MSE Loss: 2.607, valid MSE Loss: 2.944, valid RMSE Loss: 1.716, valid MAE Loss: 1.293


100%|████████████████████████████████| 508/508 [00:01<00:00, 297.88it/s]


Epoch 24 MSE Loss: 2.603, valid MSE Loss: 2.918, valid RMSE Loss: 1.708, valid MAE Loss: 1.274


100%|████████████████████████████████| 508/508 [00:01<00:00, 323.58it/s]


Epoch 25 MSE Loss: 2.601, valid MSE Loss: 2.895, valid RMSE Loss: 1.702, valid MAE Loss: 1.267


## Evaluation

In [279]:
total_test_loss = eval_MSE_loss(test_X, model)
total_test_RMSE_loss = eval_RMSE_loss(test_X, model)
total_test_MAE_loss = eval_MAE_loss(test_X, model)
print(f"test MSE Loss: {total_test_loss}, test RMSE Loss: {total_test_RMSE_loss}, test MAE Loss: {total_test_MAE_loss}")

test MSE Loss: 3.04, test RMSE Loss: 1.743, test MAE Loss: 1.311
